In [1]:
using Revise
using QuantumOptimalControl
using QuantumOptics
using LinearAlgebra
using Flux, DiffEqFlux
using PlotlyJS
using DifferentialEquations: DP5, Tsit5, Vern7, Vern9, BS3
using Random
using ProgressMeter
ProgressMeter.ijulia_behavior(:clear)

WebIO._IJuliaInit()

false

In [2]:
ω₀ = 2π*5.0
η = -2π*300*1e-3
ωlo = ω₀ + η

29.530970943744055

In [3]:
n_levels = 3
bs = FockBasis(n_levels-1)
a = destroy(bs)
ad = create(bs)
id = identityoperator(bs, bs);

In [4]:
H0 = ω₀*(ad*a) + (η/2.0)*(ad*a)*(ad*a-id)
H1 = 1.0im*(a - ad);

In [5]:
Random.seed!(1)

t0, t1 = 0.0, 4.0

Iquad(A, σ) = t -> A * exp(-0.5*(t-0.5*t1)^2 / σ^2)
Qquad(A, β, σ) = t-> β * (t-0.5*t1) * Iquad(A, σ)(t)

coeffs(params, t) = let b = params
                        [Iquad(b[1], b[3])(t)*cos(b[4]*t)+
                         Qquad(b[1], b[2], b[3])(t)*sin(b[4]*t)]
                    end

θ = [0.5, 0.5, 0.5, ωlo];

In [6]:
bcs(ps) = (abs(Iquad(ps[1], ps[3])(t0)) + abs(Iquad(ps[1], ps[3])(t1)) + 
          abs(Qquad(ps[1], ps[2], ps[3])(t0)) + abs(Qquad(ps[1], ps[2], ps[3])(t1)))
                 
cost = CostFunction((x,y)-> 1.0-real(x'*y), bcs)

CostFunction(var"#5#6"(), bcs)

In [7]:
trans = UnitaryTransform([fockstate(bs, 0), fockstate(bs, 1)], 
                         [[1.0 -1.0im];[-1.0im 1.0]]/√2);

In [8]:
tspan = (t0, t1)
H = Hamiltonian(H0, [H1], coeffs)
prob = QOCProblem(H, trans, tspan, cost);

In [9]:
@time sol = solve(prob, θ, ADAM(0.1); maxiter=200)

Progress: 100%|█████████████████████████████████████████| Time: 0:01:57
  distance:     0.013969414960976256
  constraints:  0.001796849747079937


155.559289 seconds (269.33 M allocations: 16.321 GiB, 49.68% gc time, 42.34% compilation time)


Solution{Float64}([0.8256629455996194, -3.5066885347205217, -0.4739183887742367, 30.62743556880742], [0.39663163188515316, 0.34992984400218824, 0.31289921757769756, 0.29192041813718056, 0.2853583305787027, 0.2852465174504758, 0.2855598017836281, 0.28677326317596513, 0.2899216206708214, 0.2928709362587802  …  0.013963077767183585, 0.013963640423138302, 0.013966098348027756, 0.013967604090731778, 0.013967308312732518, 0.013967563334014443, 0.013969361740646613, 0.013970612662810633, 0.013970057880980657, 0.013969414960976256], [0.0006709252558050237, 6.558915340478049e-6, 3.151757048257349e-10, 2.551588558268004e-22, 1.8130506291615267e-72, 0.0, 0.0, 1.1594508463020744e-303, 1.7211069932701165e-116, 5.265127029076157e-85  …  0.0018033331816444185, 0.0018027530017745315, 0.0018002337326818402, 0.0017986928874173572, 0.0017989950882086711, 0.0017987371477447586, 0.001796897727103228, 0.0017956204226311982, 0.0017961892558149657, 0.001796849747079937], Vector{Float64}[])

In [10]:
sol.params

4-element Vector{Float64}:
  0.8256629455996194
 -3.5066885347205217
 -0.4739183887742367
 30.62743556880742

In [11]:
plot(sol.distance_trace)

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin and template"

In [12]:
I_wf(t) = Iquad(sol.params[1], sol.params[3])(t)
Q_wf(t) = Qquad(sol.params[1], sol.params[2], sol.params[3])(t)

Q_wf (generic function with 1 method)

In [13]:
ts = collect(t0:t1/100.0:t1)
f= plot([
    scatter(x=ts, y=I_wf.(ts), name="I")
    scatter(x=ts, y=Q_wf.(ts), name="Q")
    ],
    Layout(
        xaxis_title_text="Time (ns)",
        yaxis_title_text="Frequency (GHz)",
        legend=attr(x=0, y=1,),
        font=attr(
            size=16,
        )
    )
)

data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with fields font, legend, margin, template, xaxis, and yaxis"